In [4]:
# --- Load fine-tuned model for inference on GPU ---

# 1. Install required libraries
print("Installing necessary libraries...")
!pip install -q -U transformers accelerate peft trl scipy datasets
!pip install -q -U "huggingface_hub[cli]"

# 2. Import libraries and setup environment
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

# Set environment for GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Change if needed to select GPU device
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

print("Libraries installed and environment ready for GPU.")

# 3. Login to Hugging Face using Kaggle Secrets
try:
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("HF_TOKEN")
    login(token=secret_value)
    print("\nLogged in to Hugging Face successfully.")
except Exception as e:
    print(f"\nLogin error: {e}")
    print("Make sure 'HF_TOKEN' is set in Kaggle Secrets with write access.")
    # login()  # Uncomment to login manually if needed

# 4. Set dataset and model paths
KAGGLE_DATASET_NAME = "my-medgemma-finetuned-model"  # Replace with your dataset name
model_path_in_dataset = f"/kaggle/input/farsi-tuned"
base_model_name = "google/medgemma-4b-it"

print(f"\nLoading fine-tuned model from: {model_path_in_dataset}")

# 5. Load tokenizer from fine-tuned model folder, fallback to base model tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path_in_dataset, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    print("Tokenizer loaded from fine-tuned model folder.")
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

# 6. Load base model for GPU with float32 precision and automatic device mapping
print(f"Loading base model '{base_model_name}' for GPU...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float32,  # Use float32 for GPU efficiency
    device_map="auto",          # Automatically distribute model on GPU(s)
)
print("Base model loaded for GPU.")

# 7. Apply LoRA adapters from the fine-tuned model folder
print(f"Applying LoRA adapters from {model_path_in_dataset}...")
model = PeftModel.from_pretrained(base_model, model_path_in_dataset)
print("LoRA adapters applied.")

# Set model to evaluation mode for inference
model.eval()
print("Model set to evaluation mode.")

# 8. (Optional) Merge LoRA adapters into base model for deployment (needs more RAM)
# model = model.merge_and_unload()
# print("LoRA adapters merged with base model.")

# 9. Create text generation pipeline for inference on GPU
print("\nCreating text generation pipeline...")
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,
    device_map="auto",
)

print("Fine-tuned model ready for inference!")

# 10. Test the model with a sample prompt
prompt_test = "سوال: چه عصبی در سندروم کارپل تونل به مشکل میخورد\nپاسخ:"
print(f"\n--- Test Question ---\n{prompt_test.replace('پاسخ:', '')}")
outputs_test = text_generator(
    prompt_test,
    max_new_tokens=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
generated_text_test = outputs_test[0]["generated_text"]
answer_test = generated_text_test.replace(prompt_test, "").strip()
print(f"Generated Answer: {answer_test}")


Installing necessary libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 95.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 50.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2025-07-22 15:47:14.701158: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753199235.070141      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753199235.172544      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Libraries installed and environment ready for GPU.

Logged in to Hugging Face successfully.

Loading fine-tuned model from: /kaggle/input/farsi-tuned
Tokenizer loaded from fine-tuned model folder.
Loading base model 'google/medgemma-4b-it' for GPU...


config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Base model loaded for GPU.
Applying LoRA adapters from /kaggle/input/farsi-tuned...


Device set to use cuda:0


LoRA adapters applied.
Model set to evaluation mode.

Creating text generation pipeline...
Fine-tuned model ready for inference!

--- Test Question ---
سوال: چه عصبی در سندروم کارپل تونل به مشکل میخورد

Generated Answer: نورون‌های حس‌دار که به دست می‌روند در سندروم کارپل تونل به مشکل می‌خورند.

سندروم کارپل تونل یک اختلال عصبی است که در آن عصب مدیان در مچ دست فشرده می‌شود. این عصب حس‌دار به دست می‌رود و مسئول انتقال پیام‌های حس‌دار از دست به مغز است. فشرده شدن این عصب می‌تواند باعث درد، سوزش، بی‌حسی و گز


In [5]:
prompt_test = "سوال: چه عصبی در سندروم کارپل تونل به مشکل میخورد\nپاسخ:"
print(f"\n--- Test Question ---\n{prompt_test.replace('پاسخ:', '')}")
outputs_test = text_generator(
    prompt_test,
    max_new_tokens=100,
    do_sample=False,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
generated_text_test = outputs_test[0]["generated_text"]
answer_test = generated_text_test.replace(prompt_test, "").strip()
print(f"Generated Answer: {answer_test}")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Test Question ---
سوال: چه عصبی در سندروم کارپل تونل به مشکل میخورد

Generated Answer: در سندروم کارپل تونل، عصبی که به مشکل میخورد، **نورون اپیکونیاکال** (Ulnar nerve) است.

نورون اپیکونیاکال در ناحیه مچ دست، در داخل کارپل تونل قرار دارد. فشار بر این نورون باعث ایجاد علائمی مانند گزگز، سوزن سوزن شدن، ضعف و بی‌حسی در انگشتان اشاره، میانی و قسمت بالایی انگ


In [7]:
prompt_test = "سوال: آکرومگالی چیست؟\nپاسخ:"
print(f"\n--- Test Question ---\n{prompt_test.replace('پاسخ:', '')}")
outputs_test = text_generator(
    prompt_test,
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
generated_text_test = outputs_test[0]["generated_text"]
answer_test = generated_text_test.replace(prompt_test, "").strip()
print(f"Generated Answer: {answer_test}")



--- Test Question ---
سوال: آکرومگالی چیست؟

Generated Answer: آکرومگالی یک اختلال نادر است که در آن رشد استخوان‌ها در ناحیه آکروم، که در بالای بازو و دست قرار دارد، مختل می‌شود. این اختلال باعث می‌شود که استخوان‌های بازو و دست به طور غیرطبیعی بزرگ و ضخیم شوند.
علائم آکرومگالی می‌تواند شامل موارد زیر باشد:
*   بازو و دست‌های بزرگ و ضخیم
*   سایش یا ساییدگی پوست در ناحیه آکروم
*   درد در بازو و دست
*   مشکل در حرکت دادن بازو و دست
*   تغییر شکل در ساختار استخوان
*   در موارد نادر، مشکلات در سایر قسمت‌های بدن مانند قلب و ریه‌ها
آکرومگالی معمولاً ناشی از یک تومور در غده هیپوفیز است که هورمون‌های رشد را ترشح می‌کند. تشخیص آکرومگالی معمولاً از طریق تصویربرداری پزشکی و آزمایش‌های خون انجام می‌شود. درمان آکرومگالی معمولاً شامل جراحی برای برداشتن تومور هیپوفیز و/یا درمان‌های پزشکی برای کاهش سطح هورمون‌های رشد است.

اگر فکر می‌کنید که ممکن است آکرومگالی داشته باشید، مهم است که با یک متخصص پزشکی مشورت کنید.
در اینجا چند نکته اضافی در مورد آکرومگالی وجود دارد:
*   آکرومگالی یک اختلال نادر است، ا

In [8]:
prompt_test = "سوال: علائم اولیه کم‌خونی چیست؟\nپاسخ:"
print(f"\n--- Test Question ---\n{prompt_test.replace('پاسخ:', '')}")
outputs_test = text_generator(
    prompt_test,
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
generated_text_test = outputs_test[0]["generated_text"]
answer_test = generated_text_test.replace(prompt_test, "").strip()
print(f"Generated Answer: {answer_test}")



--- Test Question ---
سوال: علائم اولیه کم‌خونی چیست؟

Generated Answer: علائم اولیه کم‌خونی می‌تواند خفیف و ناچیز باشد و به مرور زمان تشدید شود. برخی از علائم اولیه کم‌خونی عبارتند از:
* خستگی و ضعف
* سرگیجه و تنگی نفس
* سردرد
* بی‌حسی یا گزگز در دست‌ها و پاها
* کاهش اشتها
* بی‌خوابی
* مشکلات حافظه و تمرکز
* پوست شکننده و کبود شدن آسان
* لکه‌های زرد یا قهوه‌ای رنگ زیر پوست (که به آن "پلاتل هایز" می‌گویند)

اگر این علائم را تجربه می‌کنید، مهم است که به پزشک مراجعه کنید تا علت آن را تشخیص دهد و درمان مناسب را دریافت کنید.
* * *
توجه: این اطلاعات فقط برای اهداف آموزشی است و نباید به عنوان جایگزینی برای مشاوره پزشکی حرفه‌ای در نظر گرفته شود. همیشه با یک متخصص مراقبت‌های بهداشتی مشورت کنید تا در مورد هر گونه سوال یا نگرانی سلامتی خود صحبت کنید.
علائم اولیه کم‌خونی می‌تواند متفاوت باشد و همه افراد ممکن است همه این علائم را تجربه نکنند.
* * *
علائم کم خونی معمولاً به تدریج ظاهر می‌شوند و به مرور زمان بدتر می‌شوند. برخی از علائم رایج عبارتند از:
* خستگی
* سرگیجه
* تنگی نفس
* سردرد
* بی‌حسی ی

In [9]:
prompt_test = "سوال: چرا تب بالا در کودکان خطرناک است؟\nپاسخ:"
print(f"\n--- Test Question ---\n{prompt_test.replace('پاسخ:', '')}")
outputs_test = text_generator(
    prompt_test,
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
generated_text_test = outputs_test[0]["generated_text"]
answer_test = generated_text_test.replace(prompt_test, "").strip()
print(f"Generated Answer: {answer_test}")



--- Test Question ---
سوال: چرا تب بالا در کودکان خطرناک است؟

Generated Answer: تب بالا در کودکان می‌تواند به دلایل مختلفی خطرناک باشد، از جمله:

*   **افزایش خطر عفونت:** تب نشان‌دهنده فعالیت سیستم ایمنی بدن در برابر عفونت است. تب بالا می‌تواند سیستم ایمنی را بیش از حد تحریک کرده و باعث شود بدن در برابر عفونت آسیب‌پذیرتر شود.
*   **تشنج:** تب بالا می‌تواند باعث تشنج شود، که می‌تواند به اعصاب آسیب برساند و عوارض جدی داشته باشد.
*   **مشکلات قلبی:** تب بالا می‌تواند باعث مشکلات قلبی شود، مانند ضربان قلب سریع یا نامنظم.
*   **مشکلات ریوی:** تب بالا می‌تواند باعث مشکلات ریوی شود، مانند پنومونی (عفونت ریه).
*   **تومور مغزی:** در موارد نادر، تب بالا می‌تواند نشان‌دهنده وجود تومور مغزی باشد.
*   **مشکلات مغزی:** تب بالا می‌تواند باعث مشکلات مغزی شود، مانند التهاب مغزی یا مسمومیت مغزی.

بنابراین، اگر کودکی تب بالا دارد، مهم است که فوراً به پزشک مراجعه کنید. پزشک می‌تواند علت تب را تشخیص داده و درمان مناسب را تجویز کند.

در اینجا چند نکته اضافی وجود دارد:

*   تب بالا در کودکان معمولاً به د

In [10]:
prompt_test = "سوال: چگونه می‌توان از ابتلا به هپاتیت B پیشگیری کرد؟\nپاسخ:"
print(f"\n--- Test Question ---\n{prompt_test.replace('پاسخ:', '')}")
outputs_test = text_generator(
    prompt_test,
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
generated_text_test = outputs_test[0]["generated_text"]
answer_test = generated_text_test.replace(prompt_test, "").strip()
print(f"Generated Answer: {answer_test}")



--- Test Question ---
سوال: چگونه می‌توان از ابتلا به هپاتیت B پیشگیری کرد؟

Generated Answer: ابتلا به هپاتیت B قابل پیشگیری است. با رعایت نکات زیر می‌توانید از ابتلا به این بیماری جلوگیری کنید:

1. **واکسیناسیون:**
   *   **رایج‌ترین و موثرترین روش پیشگیری، تزریق واکسن هپاتیت B است.** این واکسن بسیار ایمن و موثر است و می‌تواند از 95% تا 100% از ابتلا به هپاتیت B جلوگیری کند.
   *   **نحوه تزریق:** واکسن هپاتیت B معمولاً به صورت سه دوز تزریق می‌شود. اولین دوز معمولاً در دوران بارداری (اگر مادر حامل بار باشد) یا در سنین کودکی (در صورت عدم وجود واکسن در دوران بارداری) تزریق می‌شود. دوزهای بعدی معمولاً چند هفته یا چند ماه بعد از اولین دوز تزریق می‌شوند.
   *   **توصیه‌ها:**
        *   **در دوران بارداری:** مادران باردار باید واکسن هپاتیت B را دریافت کنند.
        *   **در دوران کودکی:** کودکان باید در سنین 2، 3 و 4 سالگی واکسن هپاتیت B را دریافت کنند.
        *   **در سنین بالاتر:** افراد بالای 18 سال که قبلاً واکسن دریافت نکرده‌اند، می‌توانند واکسن هپاتیت B را دریافت کنند.

2.  **اجتن

In [11]:
prompt_test = "سوال: ارتباط بین میکروبیوم روده و اختلالات روانی چیست؟\nپاسخ:"
print(f"\n--- Test Question ---\n{prompt_test.replace('پاسخ:', '')}")
outputs_test = text_generator(
    prompt_test,
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
generated_text_test = outputs_test[0]["generated_text"]
answer_test = generated_text_test.replace(prompt_test, "").strip()
print(f"Generated Answer: {answer_test}")



--- Test Question ---
سوال: ارتباط بین میکروبیوم روده و اختلالات روانی چیست؟

Generated Answer: ارتباط بین میکروبیوم روده و اختلالات روانی یک رابطه پیچیده و دوطرفه است. به عبارت دیگر، میکروبیوم روده می‌تواند بر سلامت روان تأثیر بگذارد و در عین حال، اختلالات روانی نیز می‌توانند بر ترکیب و عملکرد میکروبیوم روده تأثیر بگذارند.

**نحوه تأثیر میکروبیوم روده بر سلامت روان:**

*   **تولید مواد مغذی:** میکروبیوم روده می‌تواند مواد مغذی مانند اسیدهای چرب آمیج (short-chain fatty acids - SCFAs) مانند اسیدهای چرب اولریک (butyric acid)، اسیدهای چرب اسیتیک (acetic acid) و اسیدهای چرب پانتوتنیک (propionic acid) را تولید کند. این مواد مغذی می‌توانند به طور مستقیم بر سیستم عصبی مرکزی (CNS) تأثیر بگذارند و در تنظیم خلق و خو، اضطراب و افسردگی نقش داشته باشند.
*   **تولید مواد شیمیایی:** میکروبیوم روده می‌تواند مواد شیمیایی مانند سروتونین، دوپامین و نوروترانسمیترهای دیگر را تولید کند که در تنظیم خلق و خو و عملکرد شناختی نقش دارند.
*   **تأثیر بر سیستم ایمنی:** میکروبیوم روده بر سیستم ایمنی بدن تأثیر می‌گ